In [ ]:
import pyreadstat
import pandas as pd

file_path = "ATP W89.sav"
df, meta = pyreadstat.read_sav(file_path)
# print(df.shape) # 164 columns

# drop clearly irrelevant
# only need columns with personal/demographic data, useful social media habits, target
# drop other survey response columns

# print(df.columns)
df = df.drop(columns=["QKEY"])
interview_data = [x for x in df.columns if ("INTERVIEW" in x or "DEVICE" in x or "FORM" in x)] # asked ChatGPT for example subsetting list based on element values
df = df.drop(columns=interview_data)

# our target: how engaged individuals were
target = "ENGACTCT_W89"
# 2 features about social media habits selected from examining meaning in questionnaire/their metadata labels
sm_features = ["CLIMSEEK_W89","FOLCLIM_W89"]

# no longer need other questionnaire data, only using ENGACTCT to measure engagement, two social media habit cols relevant, personal data
personal_data = [x for x in df.columns if ("W89" not in x)]

keep_columns = personal_data + sm_features + [target]
df = df.loc[:,keep_columns]
# print(df.columns)

# using PCA to determine PCs, most used features in those PCs, dropping others
# need at least 100 instances, drop those where don't have value for any of selected features
# modified range, examined few columns at time, view labels to see what data values contained
# for i in range(30,31):
#     print(df[df.columns[i]].value_counts())
#     print(meta.variable_value_labels.get(df.columns[i]))

def get_na_columns():
    na_counts = pd.DataFrame({"na":df.isna().apply(sum)})
    na_columns = na_counts.loc[na_counts["na"]>0]
    return na_columns

df = df.loc[df["CLIMSEEK_W89"].isna()==False]
df = df.loc[df["FOLCLIM_W89"].isna()==False]

# print(get_na_columns())
# na_columns = get_na_columns()
# for col in na_columns.index:
#     print(df[col].value_counts())
#     print(meta.variable_value_labels.get(col))
df = df.drop(columns=["F_BORN","F_PARTYLN_FINAL"]) # dropped these columns b/c high proportion of NaNs & exist at least 1 other complete col describing similar data

# print(get_na_columns()) # other cols have unique, useful info and few enough NaNs to impute

# impute missing vals with mode
impute_cols = get_na_columns().index
for i in range(len(impute_cols)):
    column = impute_cols[i]
    col_mode = df[column].mode()[0]
    df[column] = df[column].fillna(col_mode)
    df[column] = df[column].apply(lambda x: col_mode if x==99.0 else x) # treat 99.0 ("Refused") as missing val
    # print(df[column].value_counts()) # verify no 99.0s remain

# print(get_na_columns())

(13749, 164)
